In [135]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};

<IPython.core.display.Javascript object>

In [140]:
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer

from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

from functools import reduce
from itertools import combinations

In [141]:
products_df = pd.read_csv('products.csv')
orders_df = pd.read_csv('orders.csv')
#orders_products_prior_df = pd.read_csv('order_products__prior.csv')
orders_products_train_df = pd.read_csv('order_products__train.csv')

#orders_products_df = pd.concat([orders_products_prior_df, orders_products_train_df])
orders_products_df = orders_products_train_df

#orders_train_df = orders_df[orders_df['eval_set'].isin(['prior', 'train'])]
orders_train_df = orders_df[orders_df['eval_set'].isin(['train'])]
orders_test_df = orders_df[orders_df['eval_set'] == 'test']

users_orders_products_train_df = reduce(lambda l,r: pd.merge(l,r,on='order_id'), [orders_products_df, orders_train_df])
users_orders_products_test_df = reduce(lambda l,r: pd.merge(l,r,on='order_id'), [orders_products_df, orders_test_df])

In [ ]:
# feature_columns = ['order_id','product_id','add_to_cart_order','user_id','order_number','order_dow', 
#                    'order_hour_of_day','days_since_prior_order']
feature_columns = ['order_id','product_id','add_to_cart_order','user_id','order_number','order_dow', 
                   'order_hour_of_day','days_since_prior_order']
X = users_orders_products_train_df[feature_columns]
y = users_orders_products_train_df['reordered']

imputer = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

X = imputer.fit_transform(X)

In [133]:
enc = OneHotEncoder()

#d = pd.get_dummies(users_orders_products_train_df, columns=['product_id'])

#d.head()

In [111]:
my_ANN = MLPClassifier(hidden_layer_sizes=(100,10), activation= 'relu', 
                       solver='adam', alpha=1e-5, learning_rate_init = 0.01)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

my_ANN.fit(X_train, y_train)

y_predict_ann = my_ANN.predict(X_test)

score_ann = accuracy_score(y_test, y_predict_ann)

print('Accuracy of reorder prediction using ANN(Layers=3):', format(score_ann*100, '.2f'))

Accuracy of reorder prediction using ANN(Layers=3): 59.82


In [90]:
# define a range for the "number of neurons" in the hidden layer for a network with 1 hidden layer:
neuron_number = [(i,j) for i in range(1,11) for j in range(1,11)]                    

# create a dictionary for grid parameter:
param_grid = dict(hidden_layer_sizes = neuron_number)
print(param_grid,'\n')

# instantiate the model:
my_ANN = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, random_state=1, learning_rate_init = 0.1)

# creat the grid, and define the metric for evaluating the model: 
grid = GridSearchCV(my_ANN, param_grid, cv=10, scoring='accuracy')

# fit the grid (start the grid search):
grid.fit(X, y)

# view the results:
# print(grid.cv_results_)

# view the best results corresponding to the best structure of ANN:
print(grid.best_score_)
print(grid.best_params_)

{'hidden_layer_sizes': [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10)]} 

0.5962085337743035
{'hidden_layer_sizes': (1, 1)}


In [210]:
min_support = 0.004
min_confidence = 0.5

# References: https://www-users.cs.umn.edu/~kumar001/dmbook/ch6.pdf
#             https://en.wikipedia.org/wiki/Apriori_algorithm
def apriori(txn_set):
    '''
    Apriori algorithm implementation that returns the support and calculated association rules
    Returns:
        support    - dictionary of tuple of product ids and its calculated support value
        rules_list - List of calculated rules where each rule is a dictionary with lhs of rule,
                     rhs of rule and calculated confidence value for this rule.
    '''
    support_dict = {}
    itemset_dict = get_all_single_item_count(txn_set)
    txn_size = len(txn_set)
    while len(itemset_dict) > 0:
        support_dict = {**support_dict, **itemset_dict}
        c_k = get_candidate_itemsets(itemset_dict)
        if len(c_k) != 0:
            size, ht = build_hash_tree(c_k)
            for order_id in txn_set:
                item_list = txn_set[order_id]
                c_t = subset(size, ht, item_list)
                for c in c_t:
                    c_k[c] = c_k[c] + 1
        #print('candidate_set',c_k)
        itemset_dict = {k: v for k, v in c_k.items() if v/txn_size >= min_support}
        #itemset_dict = {k: v for k, v in c_k.items() if v >= min_support}
        #print('itemset_dict',c_k)
        #support_dict = {**support_dict, **itemset_dict}
        #print('support_dict',support_dict)
    return support_dict, apriori_genrules(support_dict)

def apriori_genrules(support_dict):
    '''
    Function that generates rules based on the support. If a rule like (a,b,c)->(d) has
    low confidence then the function skips processing all other rules (a,b)->(c,d)
    and (a)->(b,c,d) for efficiency.
    Confidence of each rule of form (a)->(b) is support(a)/support(b)
    '''
    rules_list = []
    for itemset_tuple in support_dict:
        itemset_list = list(itemset_tuple)
        itemset_size = len(itemset_list)
        if itemset_size >= 2:
            for i in range(itemset_size-1, 1, -1):
                lhs = tuple(itemset_list[0:i])
                rhs = tuple(itemset_list[i:itemset_size])
                conf = support_dict[itemset_tuple]/support_dict[lhs]
                if conf >= min_confidence:
                    rule_dict = {}
                    rule_dict['lhs'] = lhs
                    rule_dict['rhs'] = rhs
                    rule_dict['conf'] = conf
                    rules_list.append(rule_dict)
                else:
                    break
    return rules_list

def get_all_single_item_count(txn_set):
    '''
    Returns all unique product ids from transactions and the count of occurrence of
    each product where count of occurrence(support) is above the given min_support.
    '''
    itemset_count_dict = {}
    txn_size = len(txn_set)
    for order_id in txn_set:
        itemset = txn_set[order_id]
        for item in itemset:
            if item not in itemset_count_dict:
                itemset_count_dict[item] = 0
            itemset_count_dict[item] = itemset_count_dict[item] + 1
    return {(k,): v for k, v in itemset_count_dict.items() if v/txn_size >= min_support}
    #return {(k,): v for k, v in itemset_count_dict.items() if v >= min_support}

def get_candidate_itemsets(itemset_dict):
    '''
    Combines a given list of item set with itself and generated a new list as the candidate list
    Only items sets whose last item differ and all other last but one product ids are same are combined. 
    '''
    c_set = {}
    itemset_list = [k for k,v in itemset_dict.items()]
    itemset_size = len(itemset_list)
    for i in range(0, itemset_size-1):
        l1 = itemset_list[i]
        l1_len = len(l1)
        for j in range(i+1, itemset_size):
            l2 = itemset_list[j]
            l2_len = len(l2)
            if l1[0:l1_len-1] == l2[0:l2_len-1] and l1[-1] != l2[-1]:
                c = l1 + l2[-1:]
                c_set[tuple(sorted(c))] = 0
    return c_set
                
class Node:
    def __init__(self, key):
        self.key = key
        self.data = []
        self.children = {}
        
    def addData(self, d):
        self.data.append(d)
    
    def addChild(self, childKey):
        if childKey not in self.children:
            self.children[childKey] = Node(childKey)
        return self.children[childKey]
    
    def getChild(self, childKey):
        if childKey in self.children:
            return self.children[childKey]
        return None
    
    def isLeafNode(self):
        return len(self.children) == 0
    
    def nodeToString(self, indent):
        s = indent+'key:'+(str(self.key) if self.key is not None else 'None')+'\n'
        data = '),('.join(','.join(str(t) for t in d) for d in self.data)
        s = s+indent+'data:[('+data+')]\n'
        s = s+indent+'isLeafNode:'+str(self.isLeafNode())+'\n'
        l = len(self.children)
        if l==0:
            s=s+indent+'children:{}\n'
        else:
            s=s+indent+'children:{\n'
            for (key,val) in self.children.items():
                s=s+val.nodeToString(indent+'  ')
            s=s+indent+'}\n'
        return s
    
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
         return self.nodeToString('')
        
def build_hash_tree(c_k):
    '''
    Builds a hash tree with given candidate item set. The max number of branches at each level will be
    the size of the item set. mod(product_id, item_set_size) will assign the product id to its
    respective branch.
    '''
    hash_tree = {}
    size = 0
    for itemset in c_k:
        size = len(itemset)
        keyset = []
        for item in itemset:
            keyset.append(item % size)
        keyset = tuple(keyset)
        if keyset not in hash_tree:
            hash_tree[keyset] = []
        hash_tree[keyset].append(itemset)
    root = Node(None)
    for keyset in hash_tree:
        n = root
        for key in keyset:
            n = n.addChild(key)
        for itemset in hash_tree[keyset]:
            n.addData(itemset)        
    return size, root
      
def subset(size, ht, item_list):
    '''
    Returns all subset candidate item sets of a given item_list.
    This function that collects all possible subsets of given size in the bigger set item_list
    This function optimizes the subset generation with the knowledge of the minimum size of subset
    that should be generated. For example no of 3 item subsets of [1,2,3,4,5] are:
        123
        124
        125
        134
        135
        145
        234
        235
        245
        345
    Out of all possible subsets, the function only returns those that are part of the hash tree
    '''
    subset_tuple_list = []
    item_list = sorted(item_list)
    item_size = len(item_list)
    if(item_size >= size):
        comb_tuples = combinations(item_list, size)
        for comb_tuple in comb_tuples:
            node = ht
            for comb_item in comb_tuple:
                node = node.getChild(comb_item%size)
                if node is None:
                    break
            if node is not None and node.isLeafNode():
                if comb_tuple in node.data:
                    subset_tuple_list.append(comb_tuple)
    return subset_tuple_list

In [211]:
def prepare_txn_set():
    txn_set = {}
    for index, row in users_orders_products_train_df.iterrows():
        order_id = row['order_id']
        product_id = row['product_id']
        if order_id not in txn_set:
            txn_set[order_id] = []
        if product_id not in txn_set[order_id]:
            txn_set[order_id].append(product_id)
    return txn_set

In [ ]:
txn_set = prepare_txn_set()
support, rules = apriori(txn_set)

print(support)
print(rules)

for r in rules:
    lhs = r['lhs']
    rhs = r['rhs']
    conf = r['conf']
    s = '('
    for p in lhs:
        s = s + products_df[products_df['product_id'] == p].iloc[0]['product_name'] + ','
    s = s[:-1] + ') -> ('
    for p in rhs:
        s = s + products_df[products_df['product_id'] == p].iloc[0]['product_name'] + ','
    s = s[:-1] + ') : ' + str(format(conf, '.2f'))
    print(s)